In [2]:
# create a table for DK parcellated PET etc data
import numpy as np
import pandas as pd
import os

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian


In [10]:

# define path
base = os.path.dirname(os.getcwd())

def load_atlas(base_path):
    """
    Load different parcellation files.
    """    
    atlas= {
        'dk_fsaverage_10k': (
            os.path.join(base, 'annot', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz'),
            os.path.join(base, 'annot', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
    ),
    'dk_fsaverage_164k': (
            os.path.join(base, 'annot', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot'),
            os.path.join(base, 'annot', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
    ),
    'dk_mni': os.path.join(base, 'annot', 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
    }
    return (atlas) 

# create parcellaters 
def create_parcellaters(atlas):
    dk_fsaverage_10k = relabel_gifti(atlas['dk_fsaverage_10k'])
    parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
    
    dk_fsaverage_164k = annot_to_gifti(atlas['dk_fsaverage_164k'])  
    parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')

    parcellater_mni = Parcellater(atlas['dk_mni'], 'MNI152')
    return (parcellater_fs10k, parcellater_fs164k, parcellater_mni) 

    
# load atlas 
atlas = load_atlas(base_path)
parcellater_fs10k, parcellater_fs164k, parcellater_mni = create_parcellaters(atlas)

# rois 
rois = pd.read_csv(os.path.join(base, 'annot' ,'atlas-desikankilliany.csv'))
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()

In [11]:
annotations = list(fetch_annotation(source=['hcps1200',
                                            'raichle',
                                            'ding2010', 
                                            'finnema2016', 
                                            'dubois2015',
                                            'gallezot2010',
                                            'gallezot2017',
                                            'hillmer2016',
                                            'jaworska2020',
                                            'kaller2017',
                                            'kantonen2020',
                                            'laurikainen2018',
                                            'normandin2015',
                                            'radnakrishnan2018',
                                            'sandiego2015',
                                            'satterthwaite2014',
                                            'savli2012',
                                            'satterthwaite2014',
                                            'smith2017',
                                            'tuominen',
                                            'naganawa2020',
                                            'fazio2016',
                                            'vijay2018']).keys())

annotations.extend(fetch_annotation(source=['norgaard2021', 'beliveau2017'], space='fsaverage').keys())
annotations.extend(fetch_annotation(source='margulies2016', desc='fcgradient01', return_single=False).keys())




In [12]:
# initialize
parcellated = dict([])

# go over each annotation and parcellate depending on the space 
for (src, desc, space, den) in annotations:

    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k

    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)

    # if subcortex included remove 
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]

/Users/laurituominen/anaconda3/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:803: RuntimeWarning: invalid value encountered in divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)


In [16]:
pd.DataFrame(parcellated).to_csv(f'{base}/brain_maps/parcellated_brain_maps.csv')